# [Decision Tree Learning](https://en.wikipedia.org/wiki/Decision_tree_learning)

Decision tree learning uses a decision tree as a model to predict an observation of some attributes.

* Classification tree analysis is when the predicted outcome is the class to which the data belongs.
* Regression tree analysis is when the predicted outcome can be considered a real number (e.g. the price of a house, or a patient's length of stay in a hospital).

Data comes in records of the form:

\\((x, Y)=(x_{1},x_{2},x_{3},...,x_{k},Y)\\)

The dependent variable, \\(Y\\), is the target variable that we are trying to understand, classify or generalize. The vector x is composed of the features, \\(x_1, x_2, x_3\\) etc., that are used for that task.

There are many specific decision-tree algorithms. Notable ones include:

* ID3 (Iterative Dichotomiser 3)
* C4.5 (successor of ID3)
* CART (Classification And Regression Tree)

## [ID3 algorithm](https://en.wikipedia.org/wiki/ID3_algorithm)
In decision tree learning, ID3 (Iterative Dichotomiser 3) is an algorithm invented by Ross Quinlan used to generate a decision tree from a dataset. ID3 is the precursor to the C4.5 algorithm, and is typically used in the machine learning and natural language processing domains.

Let's use [the code](https://github.com/Claygirl/decision-trees) to learn the algorithm. [This code](https://github.com/Erikfather/Decision_tree-python) is also very useful.

In [ ]:
import numpy as np
import math
import csv

In [ ]:
def read_data(filename):

    with open(filename, 'r') as csvfile:
        datareader = csv.reader(csvfile, delimiter=',')
        headers = next(datareader)
        metadata = []
        traindata = []
        for name in headers:
            metadata.append(name)
            
        for row in datareader:
            traindata.append(row)

    return (metadata, traindata)

In [ ]:
metadata, traindata = read_data("data/playtennis.txt")

In [ ]:
metadata, traindata = read_data("data/stock.txt")

In [ ]:
class Node:
    def __init__(self, attribute):
        self.attribute = attribute
        self.children = []
        self.answer = ""
        
    def __str__(self):
        return self.attribute

In [ ]:
data = np.array(traindata)
data.shape

In [ ]:
items = np.unique(data[:, 0])
items

In [ ]:
# partition the data table into subtables by column value
# parameters:
#    data: the datatable
#    col:  the column to split
#    delete: whether to delete the column from the result
# returns
#    items: the column value set
#    dict: the dictionary of subtables by items
def subtables(data, col, delete):
    dict = {}
    items = np.unique(data[:, col])
    count = np.zeros((items.shape[0], 1), dtype=np.int32)[:, 0]
    
    for x in range(items.shape[0]):
        for y in range(data.shape[0]):
            if data[y, col] == items[x]:
                count[x] += 1
                
    for x in range(items.shape[0]):
        dict[items[x]] = np.empty((count[x], data.shape[1]), dtype="|S32")
        pos = 0
        for y in range(data.shape[0]):
            if data[y, col] == items[x]:
                dict[items[x]][pos] = data[y]
                pos += 1       
        if delete:
            dict[items[x]] = np.delete(dict[items[x]], col, 1)
        
    return items, dict

In [ ]:
items, dict = subtables(data, 1, False)

In [ ]:
dict

In [ ]:
items

In [ ]:
dict[items[2]][:, -1]

## Entropy
Entropy \\(H(S)\\) is a measure of the amount of uncertainty in the data set \\(S\\).

\\(H(S) = \sum_{x \in X }-p(x)\log2_{p(x)}\\)

In [ ]:
# compute the entropy of a collection
def entropy(S):
    X = np.unique(S)

    if X.size == 1:
        return 0
    
    probs = np.zeros((X.shape[0], 1))[:, 0]
    sums = 0
    
    for x in range(X.shape[0]):
        probs[x] = sum(S == X[x]) / (S.size * 1.0)

    for p in probs:
        sums += (-p) * math.log(p, 2)
    return sums

In [ ]:
S = data[:, -1]
S

In [ ]:
probs = np.sum(S == 'yes') / (S.size * 1.0)
probs

In [ ]:
9.0 / 14 

In [ ]:
entropy(data[:, -1])

## Information Gain

Information gain \\(IG(A)\\) is the measure of the difference in entropy from before to after the set \\(S\\) is split on an attribute \\(A\\). In other words, how much uncertainty in \\(S\\) was reduced after splitting set \\(S\\) on attribute \\(A\\).

\\(IG(S,A) = H(S) - H(S,A) \\)

\\(H(S,A) = \sum_{t \in T} p(t) H(t)\\)

* \\(H(S)\\) - Entropy of set \\(S\\)
* \\(T\\) - The subsets created from splitting set \\(S\\) by attribute \\(A\\) such that \\(S = \cup_{t \in T} t\\)
* \\(p(t)\\) - The propotion of # of elements in \\(t\\) to # of elements in \\(S\\)
* \\(H(t)\\) - Entropy of set \\(t\\)

In [ ]:
def gain_ratio(data, col):
    # items in X as keys in T
    X, T = subtables(data, col, delete=False) 

    total_size = data.shape[0]
    entropies = np.zeros((X.shape[0], 1))[:, 0]
    intrinsic = np.zeros((X.shape[0], 1))[:, 0]
    
    for t in range(X.shape[0]):
        pt = T[X[t]].shape[0] / (total_size * 1.0)
        entropies[t] = pt * entropy(T[X[t]][:, -1])
        intrinsic[t] = pt * math.log(pt, 2)
        
    return -(entropy(data[:, -1]) - sum(entropies)) / sum(intrinsic)

In [ ]:
gain_ratio(data, 0)

In [ ]:
def create_node(data, metadata):
    #TODO: 

    if (np.unique(data[:, -1])).shape[0] == 1:
        node = Node("")
        node.answer = np.unique(data[:, -1])[0]
        return node
        
    gains = np.zeros((data.shape[1] - 1, 1))
    
    for col in range(data.shape[1] - 1):
        gains[col] = gain_ratio(data, col)
        
    split = np.argmax(gains)
    
    node = Node(metadata[split])    
    metadata = np.delete(metadata, split, 0)    
    
    items, dict = subtables(data, split, delete=True)
    
    for x in range(items.shape[0]):
        child = create_node(dict[items[x]], metadata)
        node.children.append((items[x], child))
    
    return node

In [ ]:
def empty(size):
    s = ""
    for x in range(size):
        s += "   "
    return s

In [ ]:
def print_tree(node, level):
    if node.answer != "":
        print(empty(level), node.answer)
        return
        
    print(empty(level), node.attribute)
    
    for value, n in node.children:
        print(empty(level + 1), value)
        print_tree(n, level + 2)

In [ ]:
data = np.array(traindata)

In [ ]:
node = create_node(data, metadata)

In [ ]:
print_tree(node, 0)